In [1]:
# Dependencies
# ----------------------------------
import pandas as pd

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Date, DateTime

from sqlalchemy import create_engine, ForeignKey

Base = declarative_base()

# from sqlalchemy.types import Boolean, Date, DateTime, Float, Integer, Text, Time, Interval

In [2]:
msa_df = pd.read_csv('data/final_msa.csv', index_col=0)
msa_df.head()

,incident_id,incident_date,state,city_county,address,number_killed,number_injured,latitude,longitude
0,1987241,2021-04-26,Louisiana,Houma,656 Corporate Dr,0,5,36.145096,-96.002364
1,1987235,2021-04-25,Alabama,Phenix City,1700 Highway 280 Bypass,0,4,37.764028,-122.174394
2,1986593,2021-04-25,Arkansas,Little Rock,1900 E 6th St,1,3,37.719488,-122.425285
3,1986770,2021-04-25,Missouri,Kansas City,E 18th St and Vine St,1,3,41.771844,-87.672785
4,1985432,2021-04-24,Louisiana,New Orleans,100 block of Bourbon St,0,5,30.702998,-88.078683


In [3]:
msa_df['incident_date'] =  pd.to_datetime(msa_df['incident_date'])

In [4]:
msa_df.dtypes

incident_id                int64
incident_date     datetime64[ns]
state                     object
city_county               object
address                   object
number_killed              int64
number_injured             int64
latitude                 float64
longitude                float64
dtype: object

In [5]:
# class Directors(Base):
#     __tablename__ = 'directors'
#     director_id = Column(String, primary_key=True)
#     director_name = Column(String(255))

In [6]:
# Create Classes
class Msa(Base):
    __tablename__ = 'msa'
    incident_id = Column(Integer, primary_key=True)
    incident_date = Column(Date)
    state = Column(String(255))
    city = Column(String(255))
    address = Column(String(255))
    killed = Column(Integer)
    injured = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)

In [7]:
# These tables only exists in python and not in the actual database
Base.metadata.tables

immutabledict({'msa': Table('msa', MetaData(bind=None), Column('incident_id', Integer(), table=<msa>, primary_key=True, nullable=False), Column('incident_date', Date(), table=<msa>), Column('state', String(length=255), table=<msa>), Column('city', String(length=255), table=<msa>), Column('address', String(length=255), table=<msa>), Column('killed', Integer(), table=<msa>), Column('injured', Integer(), table=<msa>), Column('latitude', Float(), table=<msa>), Column('longitude', Float(), table=<msa>), schema=None)})

In [8]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
engine = create_engine("sqlite:///data/msa.sqlite")
conn = engine.connect()

In [9]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [10]:
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
# incident_id	incident_date	state	city_county	address	number_killed	number_injured	latitude	longitude

#     incident_id = Column(Integer, primary_key=True)
#     incident_date = Column(Date)
#     state = Column(String(255))
#     city = Column(String(255))
#     address = Column(String(255))
#     killed = Column(Integer)
#     injured = Column(Integer)
#     latitude = Column(Float)
#     longitude = Column(Float)

In [14]:
# Note that adding to the session does not update the table. It queues up those queries.
# Adding data to directors table from directors csv file
for index, name in msa_df.iterrows():
    session.add(Msa(incident_id=name['incident_id'], incident_date=name['incident_date'], state=name['state'], \
                   city=name['city_county'], address=name['address'], killed=name['number_killed'], \
                   injured=name['number_injured'], latitude=name['latitude'], \
                   longitude=name['longitude']
                   ))

In [15]:
session.new

IdentitySet([<__main__.Msa object at 0x000001904F41C160>, <__main__.Msa object at 0x0000019052311E80>, <__main__.Msa object at 0x0000019051D675C0>, <__main__.Msa object at 0x0000019051D675F8>, <__main__.Msa object at 0x0000019051D67668>, <__main__.Msa object at 0x0000019051D676D8>, <__main__.Msa object at 0x0000019051D67748>, <__main__.Msa object at 0x0000019051D677B8>, <__main__.Msa object at 0x0000019051D67828>, <__main__.Msa object at 0x0000019051D67898>, <__main__.Msa object at 0x0000019051D67908>, <__main__.Msa object at 0x0000019051D67978>, <__main__.Msa object at 0x0000019051D679E8>, <__main__.Msa object at 0x0000019051D67A58>, <__main__.Msa object at 0x0000019051D67AC8>, <__main__.Msa object at 0x0000019051D67B38>, <__main__.Msa object at 0x0000019051D67BA8>, <__main__.Msa object at 0x0000019051D67C18>, <__main__.Msa object at 0x0000019051D67C88>, <__main__.Msa object at 0x0000019051D67CF8>, <__main__.Msa object at 0x0000019051D67D68>, <__main__.Msa object at 0x0000019051D67EF0

In [16]:
session.commit()

In [ ]:
# session.rollback()

In [17]:
session.close()